# Práctica 1 - Algoritmo BPE

- Martínez Ostoa Néstor I.
- Procesamiento de Lenguaje Natural
- IeC - FI - UNAM

--- 
**Objetivo**: Preprocesar un corpus a partir de métodos basados en lenguajes formales y tokenizarlo en subpalabras

De manera general, hay que seguir los siguientes pasos: 

1. Selección del corpus
2. Limpieza del corpus
3. Algoritmo BPE

## Selección del corpus

- Escoger un corpus de cualquier idioma y de un tamãno mayor a $10000$

--- 

**Corpus elegido:** Don't Patronize Me! dataset ([link](https://github.com/Perez-AlmendrosC/dontpatronizeme))

- Este corpus contiene $10,637$ párrafos extraídos de artículos de noticias con el objetivo principal de realizar un análisis para detectar lenguaje condescendiente (*patronizing and condescending language PCL*) en grupos socialmente vulnerables (refugiados, familias pobres, personas sin casa, etc)
- Cada uno de estos párrafos están anotados con etiquetas que indican el tipo de lenguaje PCL que se encuentra en él (si es que está presente). Los párrafos se extrajeron del corpus [News on Web (NOW)](https://www.english-corpora.org/now/)
- [Link al paper principal](https://aclanthology.org/2020.coling-main.518/)

---

**Estructura del corpus**

- De manera general, el dataset contiene párrafos anotados con una etiqueta con valores entre $0$ y $4$ que indican el nivel de lenguaje PCL presente
- Cada instancia del dataset está conformada de la siguiente manera:
    - ```<doc-id>```: id del documento dentro del corpus NOW
    - ```<keyword>```: término de búsqueda utilizado para extraer textos relacionados con una comunidad en específico
    - ```<country-code>```: código de dos letras ISO Alpha-2
    - ```<paragraph>```: párrafo perteneciente al ```<keyword>```
    - ```<label>```: entero que indica el nivel de PCL presente

In [1]:
import string
import re

import nltk
from nltk.corpus import stopwords

import pandas as pd

In [2]:
path = "../dontpatronizeme_v1.3/dontpatronizeme_pcl.tsv"

cols = ["doc_id", "keyword", "country-code", "paragraph", "label"]
df = pd.read_csv(path, sep='\t', skiprows=4, names=cols)

df.head()

,doc_id,keyword,country-code,paragraph,label
0,@@23953477,in-need,in,The ones in need of constant medical care are ...,0
1,@@4703096,immigrant,jm,NBC and Spanish-language Univision both declin...,0
2,@@25567226,in-need,hk,A second T-Home project is being launched in t...,0
3,@@1824078,poor-families,tz,Camfed would like to see this trend reversed ....,4
4,@@1921089,refugee,tz,Kagunga village was reported to lack necessary...,0


## Limpieza del corpus

- Eliminar signos de puntuación, de interrogación, admiración y elementos no léxicos y en general elementos ruidosos

--- 

**Valores nulos**

Comenzamos observando un panorama general del dataset para encontrar posibles valores nulos

In [3]:
df[df.isna().any(axis=1)]

,doc_id,keyword,country-code,paragraph,label
5744,@@16852855,migrant,ke,NaN,0


Viendo que el dataset cuenta con un párrafo nulo, lo podemos eliminar dado que no contamos con acceso a ese párrafo

In [4]:
df = df.dropna()
df[df.isna().any(axis=1)]

,doc_id,keyword,country-code,paragraph,label


**Construcción del corpus**

Una vez que eliminamos valores nulos, podemos pasar a la construcción del corpus. En una etapa inicial, el corpus será un dataframe de Pandas en donde cada elemento corresponderá con un párrafo del dataset original

In [5]:
def build_corpus(df):
    corpus = []
    for _, row in df.iterrows():
        p = row["paragraph"]
        corpus.append(p)
    return pd.DataFrame({'paragraph': corpus})

In [6]:
corpus_df = build_corpus(df)
print("Number of documents:", "{:,}".format(corpus_df.shape[0]))
corpus_df.head()

Number of documents: 10,059


,paragraph
0,The ones in need of constant medical care are ...
1,NBC and Spanish-language Univision both declin...
2,A second T-Home project is being launched in t...
3,Camfed would like to see this trend reversed ....
4,Kagunga village was reported to lack necessary...


**Conteo de tokens**

Verificamos la cantidad de tokens presentes en el corpus

In [7]:
def count_tokens(corpus):
    """
    corpus: Pandas DataFrame
    """
    token_count = 0
    for _, row in corpus.iterrows():
        tokens = row["paragraph"].split(' ')
        token_count += len(tokens)
    return token_count

In [8]:
token_count = count_tokens(corpus_df)
print("Number of tokens:", "{:,}".format(token_count))

Number of tokens: 513,753


**Limpieza de un documento ejemplo**

- Eliminación de espacios en blanco
- Eliminación de caracteres especiales
- Eliminación de stopwords

In [9]:
def clean_whitespaces(doc):
    new_doc = doc.strip()
    return re.sub("\s+", " ", new_doc)

def clean_punctuation(doc):
    new_doc = doc.replace('-', ' ')
    new_doc = doc.replace('/', ' ')
    new_doc = "".join([w for w in new_doc if w not in string.punctuation])
    new_doc = clean_numbers(new_doc)
    return clean_whitespaces(new_doc)

def clean_numbers(doc):
    doc = doc.replace('1', ' ')
    doc = doc.replace('2', ' ')
    doc = doc.replace('3', ' ')
    doc = doc.replace('4', ' ')
    doc = doc.replace('5', ' ')
    doc = doc.replace('6', ' ')
    doc = doc.replace('7', ' ')
    doc = doc.replace('8', ' ')
    doc = doc.replace('9', ' ')
    doc = doc.replace('0', ' ')
    return doc

stop_words = set(stopwords.words('english'))
stop_words.remove('no')
def clean_stopwords(doc):
    doc = doc.lower().split(' ')
    new_p = [w for w in doc if w not in stop_words]
    new_doc = " ".join(new_p)
    return clean_whitespaces(new_doc)

def clean_document(doc):
    new_doc = clean_punctuation(doc)
    new_doc = clean_stopwords(new_doc)
    return new_doc

In [10]:
# Elección de un documento
doc = corpus_df.iloc[30, 0]
print(f"Document of choice:\n-------------------\n{doc}\n")

# Documento limpio
new_doc = clean_document(doc)
print(f"Clean document:\n-----------------\n{new_doc}")

Document of choice:
-------------------
Being part of a wider movement on protecting the human rights of vulnerable people and advocating for more effective responses from governments and other regulatory agencies

Clean document:
-----------------
part wider movement protecting human rights vulnerable people advocating effective responses governments regulatory agencies


**Limpieza de todo el corpus**

In [11]:
clean_docs = []
for idx, row in corpus_df.iterrows():
    p = row['paragraph']
    #print(f"Current doc[{idx}]:\n---------------\n{p}\n")
    clean_doc = clean_document(p)
    #print(f"Cleaned doc:\n------------\n{clean_doc}\n\n")
    clean_docs.append(clean_doc)

clean_corpus_df = pd.DataFrame({'paragraph': clean_docs})
clean_corpus_df.head()

,paragraph
0,ones need constant medical care kept admitted ...
1,nbc spanishlanguage univision declined air sho...
2,second thome project launched third quarter on...
3,camfed would like see trend reversed would lik...
4,kagunga village reported lack necessary social...


**Construcción del corpus final**

- El corpus final será una lista de palabras

In [12]:
def build_final_corpus(df):
    corpus = dict()
    for _, row in df.iterrows():
        paragraph = row['paragraph'].split(' ')
        
        # Iteración para obtener palabras y su frecuencia asociada
        for word in paragraph:
            if word not in corpus:
                corpus[word] = 0
            corpus[word] += 1

    # Construcción del dataframe : <palabra, frecuencia>
    df = pd.DataFrame({'word': list(corpus.keys()), 'frequency': list(corpus.values())})
    df = df.sort_values(by='frequency', ascending=False)
    return df

In [13]:
corpus_df = build_final_corpus(clean_corpus_df)
print("Different words:", "{:,}".format(corpus_df.shape[0]))
corpus_df.head()

Different words: 30,741


,word,frequency
233,said,2174
132,people,1858
101,women,1769
1,need,1354
16,families,1347


## Algoritmo BPE

- Aplicar el algoritmo BPE al corpus para obtener subpalabras:
    - Formar un vocabulario inicial: cada palabra se asocia a la cadena de subpalabras
    - Seleccionar el número de iteraciones $k$ que mejor se adapte al corpus elegido
    - Obtener el vocabulario final: cada palabra se asocia a la cadena de subpalabras
    - Sustituir en el corpus las palabras por la tokenización en subpalabras obtenidas
   

### Entrada
- Lista de palabras (bajo un alfabeto $\Sigma$) y sus frecuencias.
- Número $k$ de iteraciones

In [14]:
def get_Sigma(df):
    """
    df: Pandas DataFrame with words in one of its colums
    """
    sigma = set()
    for _, row in df.iterrows():
        word = row['word']
        for letter in word:
            sigma.add(letter)
    
    return sorted(list(sigma))

In [15]:
corpus_df = build_final_corpus(clean_corpus_df)
#corpus_df = corpus_df.iloc[500:510, :]
sigma = get_Sigma(corpus_df)
k = 5

print(f"K: {k}, Sigma: {len(sigma)} elements")
print(f"\n{sigma}\n")
corpus_df.head()

K: 5, Sigma: 26 elements

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']



,word,frequency
233,said,2174
132,people,1858
101,women,1769
1,need,1354
16,families,1347


### Inicialización

- Se indican los símbolos del alfabeto $\Sigma$ presentes en las palabras
- Separación de cada palabra en su lista de símbolos: ```said``` $\rightarrow$ ```s a i d```

In [16]:
print(f"Symbols in Sigma ({len(sigma)}):\n\n{sigma}")

Symbols in Sigma (26):

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [17]:
def split_words(df):
    splitted_words = []
    for _, row in df.iterrows():
        word = row['word']
        
        #Split word into a list of its symbols
        splitted_words.append(list(word))
    
    df['word'] = splitted_words
    df = df.rename(columns={'word': 'symbols'})
    return df

In [18]:
corpus_df = split_words(corpus_df)
corpus_df.head()

,symbols,frequency
233,"[s, a, i, d]",2174
132,"[p, e, o, p, l, e]",1858
101,"[w, o, m, e, n]",1769
1,"[n, e, e, d]",1354
16,"[f, a, m, i, l, i, e, s]",1347


### Inducción
- Por $k$ iteraciones hacer:
    1. Se obtienen los pares de símbolos $[a_i,b_j]$ y sus frecuencias $f([a_i,b_j])$
    2. Se obtiene $$[a,b]=\arg\max_{a_i,b_j}\{f([a_i,b_j]):a_i,b_j\in\Sigma\}$$
    3. Se hace el reemplazo por el símbolo $ab$ en cada palabra del vocabulario: $$[a,b]\mapsto ab$$
    4. Se agrega el símbolo $ab$ al alfabeto $\Sigma$
   

In [19]:
def get_pairs(df):
    pairs_dict = dict()
    for _, row in df.iterrows():
        symbols = row['symbols']
        frequency = row['frequency']
        
        pairs = []
        for i in range(1, len(symbols)):
            pairs.append((symbols[i-1], symbols[i]))
            
        for pair in pairs:
            if pair not in pairs_dict:
                pairs_dict[pair] = 0
            pairs_dict[pair] += frequency
    
    df = pd.DataFrame({
        'pair': list(pairs_dict.keys()), 
        'frequency': list(pairs_dict.values())
    })
    return df

def get_most_frequent_pair(pairs_df):
    pairs_df_sorted = pairs_df.sort_values(by='frequency', ascending=False)
    return pairs_df_sorted.iloc[0, 0]

def update_corpus(df, pair_to_update):
    symbols_updated = []
    for idx, row in df.iterrows():
        symbols = row['symbols']
        
        sym_idx = 0
        joined_symbols = []
        i = 1
        while (i < len(symbols)):
            if pair_to_update == (symbols[i-1], symbols[i]):
                js = pair_to_update[0] + pair_to_update[1]
                joined_symbols.append(js)
                sym_idx = i
                i += 2
            else:
                joined_symbols.append(symbols[i-1])
                i += 1
        if sym_idx != len(symbols)-1 and len(symbols) > 0:
            joined_symbols.append(symbols[len(symbols)-1])
            
        symbols_updated.append(joined_symbols)
    
    return pd.DataFrame({
        'symbols':symbols_updated,
        'frequency':df['frequency']
   })
        

In [20]:
corpus_df.head()

,symbols,frequency
233,"[s, a, i, d]",2174
132,"[p, e, o, p, l, e]",1858
101,"[w, o, m, e, n]",1769
1,"[n, e, e, d]",1354
16,"[f, a, m, i, l, i, e, s]",1347


In [21]:
for i in range(k):
    print(f"------------{i}-----------------")
    pairs_df = get_pairs(corpus_df)
    mfp = get_most_frequent_pair(pairs_df)
    print(f"Most frequent pair: {mfp}\n")
    corpus_df = update_corpus(corpus_df, mfp)

------------0-----------------
Most frequent pair: ('i', 'n')

------------1-----------------
Most frequent pair: ('e', 'r')

------------2-----------------
Most frequent pair: ('e', 's')

------------3-----------------
Most frequent pair: ('e', 'n')

------------4-----------------
Most frequent pair: ('o', 'n')



In [22]:
corpus_df.head()

,symbols,frequency
233,"[s, a, i, d]",2174
132,"[p, e, o, p, l, e]",1858
101,"[w, o, m, en]",1769
1,"[n, e, e, d]",1354
16,"[f, a, m, i, l, i, es]",1347


### Salida 
- Se obtiene un nuevo vocabulario $\Sigma'$ donde se indican las sub-palabras más frecuentes


In [23]:
sigma_p_dict = dict()
for _, row in corpus_df.iterrows():
    symbols = row['symbols']
    freq = row['frequency']
    for s in symbols:
        if s not in sigma_p_dict:
            sigma_p_dict[s] = 0
        sigma_p_dict[s] += freq
    
sigma_p = sorted(list(sigma_p_dict.keys()))
print(f"Sigma P ({len(sigma_p)})\n{sigma_p}")

sigma_p_df = pd.DataFrame({
    'symbol': sigma_p,
    'frequency': list(sigma_p_dict.values())
})
sigma_p_df = sigma_p_df.sort_values(by='frequency', ascending=False)
sigma_p_df.head()

Sigma P (31)
['a', 'b', 'c', 'd', 'e', 'en', 'er', 'es', 'f', 'g', 'h', 'i', 'in', 'j', 'k', 'l', 'm', 'n', 'o', 'on', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


,symbol,frequency
1,b,135429
5,en,133148
16,m,112524
2,c,105929
0,a,99177


In [24]:
corpus_df

,symbols,frequency
233,"[s, a, i, d]",2174
132,"[p, e, o, p, l, e]",1858
101,"[w, o, m, en]",1769
1,"[n, e, e, d]",1354
16,"[f, a, m, i, l, i, es]",1347
...,...,...
17756,"[s, o, l, a, n, a]",1
17757,"[c, on, s, i, d, er, a, b, l, y]",1
17759,"[l, o, w, g, r, o, w, t, h]",1
17760,"[f, o, u, r, p, a, r, t]",1
